In [ ]:
import streamlit as st
import oracledb
from queue import Queue
from typing import List, Dict
import time
from dataclasses import dataclass
from concurrent.futures import ThreadPoolExecutor

# Database Configuration Class
@dataclass
class DBConfig:
    username: str
    password: str
    host: str
    port: int
    service_name: str
    
    def get_dsn(self) -> str:
        return f"{self.host}:{self.port}/{self.service_name}"

# Oracle Year Partitioned Query Class
class OracleYearPartitionedQuery:
    def __init__(self, db_config: DBConfig, start_year: int = 2006, end_year: int = 2023, thick_mode: bool = False):
        self.db_config = db_config
        self.start_year = start_year
        self.end_year = end_year
        self.result_queue = Queue()
        
        if thick_mode:
            oracledb.init_oracle_client()
    
    def get_connection(self):
        return oracledb.connect(
            user=self.db_config.username,
            password=self.db_config.password,
            dsn=self.db_config.get_dsn()
        )
    
    def process_year(self, base_query: str, year: int) -> List[Dict]:
        wrapped_query = f"""
            WITH filtered_data AS (
                {base_query}
            )
            SELECT *
            FROM filtered_data
            WHERE YR = {year}
        """
        results = []
        try:
            with self.get_connection() as connection:
                with connection.cursor() as cursor:
                    cursor.execute(wrapped_query)
                    columns = [col[0] for col in cursor.description]
                    
                    batch_size = 10000
                    total_rows = 0
                    while True:
                        rows = cursor.fetchmany(batch_size)
                        if not rows:
                            break
                        batch_results = [dict(zip(columns, row)) for row in rows]
                        results.extend(batch_results)
                        total_rows += len(batch_results)
                        st.write(f"Year {year}: Processed {total_rows} rows so far...")
                            
            st.write(f"Completed Year {year}: Total {len(results)} records")
            
        except Exception as e:
            st.error(f"Error processing year {year}: {str(e)}")
            raise
        
        self.result_queue.put((year, results))
        return results

    def execute_parallel(self, query: str, max_concurrent_threads: int = 4) -> List[Dict]:
        years = list(range(self.start_year, self.end_year + 1))
        total_years = len(years)
        
        st.write(f"Starting parallel execution for years {self.start_year} to {self.end_year}")
        st.write(f"Using maximum {max_concurrent_threads} concurrent threads")
        
        start_time = time.time()
        
        with ThreadPoolExecutor(max_workers=max_concurrent_threads) as executor:
            futures = [
                executor.submit(self.process_year, query, year)
                for year in years
            ]
            
            for future in futures:
                try:
                    future.result()
                except Exception as e:
                    st.error(f"Thread execution failed: {str(e)}")
        
        all_results = []
        temp_results = []
        
        while not self.result_queue.empty():
            temp_results.append(self.result_queue.get())
        
        temp_results.sort(key=lambda x: x[0])
        
        st.write("\nProcessing Summary:")
        st.write("-" * 40)
        for year, results in temp_results:
            st.write(f"Year {year}: {len(results):,} records")
            all_results.extend(results)
        
        end_time = time.time()
        total_time = end_time - start_time
        
        st.write("\nPerformance Metrics:")
        st.write("-" * 40)
        st.write(f"Total Processing Time: {total_time:.2f} seconds")
        st.write(f"Average Time per Year: {total_time/total_years:.2f} seconds")
        st.write(f"Total Records: {len(all_results):,}")
        
        return all_results

# Streamlit App Interface
def main():
    st.title("Oracle Year Partitioned Query App")

    # Sidebar configuration inputs
    st.sidebar.header("Database Configuration")
    username = st.sidebar.text_input("Username", value="your_username")
    password = st.sidebar.text_input("Password", value="your_password", type="password")
    host = st.sidebar.text_input("Host", value="your_host")
    port = st.sidebar.number_input("Port", value=1521)
    service_name = st.sidebar.text_input("Service Name", value="your_service")
    
    # Sidebar for query and year range
    start_year = st.sidebar.number_input("Start Year", min_value=2000, max_value=2023, value=2006)
    end_year = st.sidebar.number_input("End Year", min_value=2000, max_value=2023, value=2023)
    max_threads = st.sidebar.number_input("Max Concurrent Threads", min_value=1, max_value=10, value=4)
    
    base_query = st.text_area("SQL Query", value="""
        SELECT t.*, EXTRACT(YEAR FROM your_date_column) as YR
        FROM your_table t
        WHERE your_conditions = 1
    """)

    # Initialize DB config
    db_config = DBConfig(username, password, host, port, service_name)

    if st.button("Execute Query"):
        # Initialize query processor
        threaded_query = OracleYearPartitionedQuery(
            db_config=db_config,
            start_year=start_year,
            end_year=end_year
        )

        # Execute query
        results = threaded_query.execute_parallel(base_query, max_concurrent_threads=max_threads)
        
        # Optionally display results in Streamlit
        if results:
            df = pd.DataFrame(results)
            st.dataframe(df)

# Run the Streamlit app
if __name__ == "__main__":
    main()
